OUTRA OPCAO

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files
import os
import matplotlib.pyplot as plt

# Carregar o modelo ResNet50 pré-treinado
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    return features

def calculate_similarity(features1, features2):
    return cosine_similarity(features1.reshape(1, -1), features2.reshape(1, -1))

# Fazer o upload das imagens
uploaded = files.upload()

# Listar os nomes das imagens carregadas
image_paths = list(uploaded.keys())
print("Imagens carregadas:", image_paths)

# Extrair características de todas as imagens
features_list = [extract_features(img_path) for img_path in image_paths]

# Dicionário para armazenar as recomendações
recommendations = {}

# Comparar cada imagem com todas as outras
for i, img_path in enumerate(image_paths):
    similar_images = []
    for j, other_img_path in enumerate(image_paths):
        if i != j:  # Não comparar a imagem com ela mesma
            similarity = calculate_similarity(features_list[i], features_list[j])
            if similarity[0][0] > 0.5:  # Similaridade maior que 50%
                similar_images.append((other_img_path, similarity[0][0]))
    recommendations[img_path] = similar_images

# Função para exibir imagens
def display_images(img_path, similar_images):
    plt.figure(figsize=(10, 5))

    # Exibir a imagem original
    plt.subplot(1, len(similar_images) + 1, 1)
    img = image.load_img(img_path)
    plt.imshow(img)
    plt.title("Original: " + img_path)
    plt.axis('off')

    # Exibir as imagens recomendadas
    for idx, (similar_img, similarity) in enumerate(similar_images):
        plt.subplot(1, len(similar_images) + 1, idx + 2)
        img = image.load_img(similar_img)
        plt.imshow(img)
        plt.title(f"Similar: {similar_img}\nSimilaridade: {similarity:.2f}")
        plt.axis('off')

    plt.show()

# Mostrar as recomendações e exibir as imagens
for img_path, similar_images in recommendations.items():
    print(f"\nRecomendações para {img_path}:")
    if similar_images:
        display_images(img_path, similar_images)
    else:
        print("- Nenhuma imagem com similaridade > 80% encontrada.")